In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
!nvidia-smi

Sat Apr 10 19:22:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    29W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd '/content/drive/MyDrive/Colab Notebooks/RL'

/content/drive/MyDrive/Colab Notebooks/RL


In [8]:
!pip3 install gym

In [9]:
!pip install tensorboardX

     |████████████████████████████████| 122kB 8.4MB/s 


In [10]:
print("our new code!")
from lib import wrappers
from lib import dqN_model2

import argparse
import time
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.optim as optim


from tensorboardX import SummaryWriter


DEFAULT_ENV_NAME = "PongNoFrameskip-v4"
MEAN_REWARD_BOUND = 19.0

GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
REPLAY_START_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000

EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_START = 1.0
EPSILON_FINAL = 0.01

Experience = collections.namedtuple('Experience', field_names=[
                                    "state", "action", "reward", "done", "next_states"])


class ExperienceBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)

    def append(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, dones, next_states = zip(
            *[self.buffer[idx] for idx in indices])

        return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), np.array(dones, dtype=np.uint8), np.array(next_states)


class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()

    def _reset(self):
        self.state = self.env.reset()
        self.total_reward = 0.0

    @torch.no_grad()
    def play_step(self, net, epsilon=0.0, device="cuda"):
        done_reward = None

        if np.random.random() < epsilon:
            action = self.env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v = net(state_v)  # shape-> [1,n_actions]
            _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())

        new_state, reward, is_done, _ = self.env.step(action)
        self.total_reward += reward
        exp = Experience(self.state, action, reward, is_done, new_state)
        self.exp_buffer.append(exp)
        self.state = new_state
        if is_done:
            done_reward = self.total_reward
            self._reset()
        return done_reward


def calc_loss(batch, net, tgt_net, device="cuda"):
    states, actions, rewards, dones, next_states = batch

    states_v = torch.tensor(states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    state_action_values = net(states_v).gather(
        1, actions_v.unsqueeze(-1)).squeeze(-1)

    next_states_values = tgt_net(next_states_v).max(1)[0]
    next_states_values[done_mask] = 0.0

    next_states_values = next_states_values.detach()

    expected_state_action_values = rewards_v + next_states_values * GAMMA

    return nn.MSELoss()(state_action_values, expected_state_action_values)

our new code!


In [11]:
device = torch.device("cuda")

env = wrappers.make_env(DEFAULT_ENV_NAME)
net = dqN_model2.DQN(env.observation_space.shape,
                      env.action_space.n).to(device)
tgt_net = dqN_model2.DQN(env.observation_space.shape,
                          env.action_space.n).to(device)

writer = SummaryWriter(comment="-" + DEFAULT_ENV_NAME)

buffer = ExperienceBuffer(REPLAY_SIZE)
agent = Agent(env, buffer)
epsilon = EPSILON_START
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
total_rewards = []
frame_idx = 0
ts_frame = 0
ts = time.time()
best_mean_reward = None

while True:

    reward = agent.play_step(net, epsilon, device)
    if len(buffer) < REPLAY_START_SIZE:
        print("\033[F\033[  WWarning! buffering: " +
              str(round(len(buffer)/REPLAY_START_SIZE*100, 2))+"%")
        continue
    frame_idx += 1
    epsilon = max(EPSILON_FINAL, EPSILON_START -
                  frame_idx/EPSILON_DECAY_LAST_FRAME)
    if reward is not None:
        total_rewards.append(reward)
        speed = (frame_idx-ts_frame) / (time.time()-ts)
        ts_frame = frame_idx
        ts = time.time()
        mean_reward = np.mean(total_rewards[-100:])
        print("{} done {} games, mean reward {}, eps {}, speed {} f/s"
              .format(frame_idx, len(total_rewards), round(mean_reward, 3), round(epsilon, 3), round(speed, 3)))
        writer.add_scalar("epsilon", round(epsilon, 3), frame_idx)
        writer.add_scalar("speed", speed, frame_idx)
        writer.add_scalar("reward_100", round(mean_reward, 3), frame_idx)
        writer.add_scalar("reward", round(reward, 3), frame_idx)

        if best_mean_reward is None or best_mean_reward < mean_reward:
            torch.save(net.state_dict(), DEFAULT_ENV_NAME + "-best.dat")
            if best_mean_reward is not None:
                print("Best mean reward updated %.3f -> %.3f, model saved" %
                      (best_mean_reward, mean_reward))
            best_mean_reward = mean_reward
        if mean_reward > MEAN_REWARD_BOUND:
            print("Solved in %d frames!" % frame_idx)
            break

    if frame_idx % SYNC_TARGET_FRAMES == 0:
        tgt_net.load_state_dict(net.state_dict())
    optimizer.zero_grad()
    batch = buffer.sample(BATCH_SIZE)
    loss_t = calc_loss(batch, net, tgt_net, device=device)
    loss_t.backward()
    optimizer.step()
writer.close()

流式输出内容被截断，只能显示最后 5000 行内容。
[  WWarning! buffering: 56.56%
[  WWarning! buffering: 56.57%
[  WWarning! buffering: 56.58%
[  WWarning! buffering: 56.59%
[  WWarning! buffering: 56.6%
[  WWarning! buffering: 56.61%
[  WWarning! buffering: 56.62%
[  WWarning! buffering: 56.63%
[  WWarning! buffering: 56.64%
[  WWarning! buffering: 56.65%
[  WWarning! buffering: 56.66%
[  WWarning! buffering: 56.67%
[  WWarning! buffering: 56.68%
[  WWarning! buffering: 56.69%
[  WWarning! buffering: 56.7%
[  WWarning! buffering: 56.71%
[  WWarning! buffering: 56.72%
[  WWarning! buffering: 56.73%
[  WWarning! buffering: 56.74%
[  WWarning! buffering: 56.75%
[  WWarning! buffering: 56.76%
[  WWarning! buffering: 56.77%
[  WWarning! buffering: 56.78%
[  WWarning! buffering: 56.79%
[  WWarning! buffering: 56.8%
[  WWarning! buffering: 56.81%
[  WWarning! buffering: 56.82%
[  WWarning! buffering: 56.83%
[  WWarning! buffering: 56.84%
[  WWarning! buffering: 56.85%
[  WWarning! bu

In [ ]:
!python3 play.py -m='/content/drive/MyDrive/Colab Notebooks/RL/PongNoFrameskip-v4-best.dat' -e="PongNoFrameskip-v4" -r='/content/drive/MyDrive/Colab Notebooks/RL/record'